In [1]:
class Descriptor:
    def __init__(self, name=None, default=None):
        self.name = name
        self.default = default
        
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value
        
    def __get__(self, instance, objtype):
        if self.name not in instance.__dict__:
            instance.__dict__[self.name] = self.default
        return instance.__dict__[self.name]
    
    def __delete__(self, instance):
        raise AttributeError("Can't delete")

class Typed(Descriptor):
    type_ = object
    def __set__(self, instance, value):
        if not isinstance(value, self.type_):
            raise TypeError('Expected %s' % self.type_)
        super().__set__(instance, value)
        
class Numeric(Typed):
    extra_methods = ['gt', 'gte']
    
class Integer(Numeric):
    type_ = int
    
class Float(Numeric):
    type_ = float
    extra_methods = Numeric.extra_methods + ['isclose']
    
class String(Typed):
    type_ = str
    extra_methods = ['startwith', 'endwith', 'contains']
    
    def startwith(instance_value, value):
        return instance_value.startwith(value)
    
    def contains(instance_value, value):
        return value in instance_value

In [13]:
class ModelMeta(type):
    def __new__(metacls, clsname, bases=None, clsdict=None):
        cls = super().__new__(metacls, clsname, bases, clsdict)
        extended_attrs = {}
        for attr_name, attr_value in cls.__dict__.items():
            print(attr_name, attr_value, attr_value.__class__.__bases__)
            if isinstance(attr_value, (String, Integer, Float)):
                extended_attrs.update({
                    f'{attr_name}__{extra_method}':lambda self, value: None
                    for extra_method in attr_value.extra_methods
                })
        for attr_name, attr_value in extended_attrs.items():
            setattr(cls, attr_name, attr_value)
        return cls


In [15]:
class Attribute:

    def __init__(self, initval=None, name='var'):
        self.val = initval
        self.name = name

    def __get__(self, obj, objtype):
        print('Retrieving', self.name, id(obj))
        return obj.__dict__[self.name]  # self.val

    def __set__(self, obj, val):
        print('Updating', self.name, id(obj))
        self.val = val
        obj.__dict__[self.name] = val

    def __delete__(self, obj):
        # print('Deleting', self.name, id(obj))
        self.val = None


class A:
    attr = Attribute(name='attr')
    # attr = 'DEMO'
    
a = A()
a.attr=10

Updating attr 139896951349408


In [21]:
class RangeInteger(object):
    def __init__(self, name, min_value=0, max_value=100):
        self.name = name
        self.min_value = min_value
        self.max_value = max_value

    def __get__(self, instance, owner):
        return self.value

    def __set__(self, instance, value):
        if self.min_value <= value <= self.max_value:
            self.value = value
        else:
           raise Exception('Out of range')

class Employee(metaclass=ModelMeta):
    first_name = String(default='John')
    last_name = String()
    age = Integer(default=42)
    salary = Float()
    phone_number = String()
    kpi_score = RangeInteger(name='kpi_score', min_value=0, max_value=100)
    
emp = Employee()
emp.kpi_score = 101
print(emp.kpi_score)


__module__ __main__ (<class 'object'>,)
first_name <__main__.String object at 0x7f3c367096d0> (<class '__main__.Typed'>,)
last_name <__main__.String object at 0x7f3c36709d30> (<class '__main__.Typed'>,)
age <__main__.Integer object at 0x7f3c36709580> (<class '__main__.Numeric'>,)
salary <__main__.Float object at 0x7f3c36709c40> (<class '__main__.Numeric'>,)
phone_number <__main__.String object at 0x7f3c367090a0> (<class '__main__.Typed'>,)
kpi_score <__main__.RangeInteger object at 0x7f3c36709e50> (<class 'object'>,)
__dict__ <attribute '__dict__' of 'Employee' objects> (<class 'object'>,)
__weakref__ <attribute '__weakref__' of 'Employee' objects> (<class 'object'>,)
__doc__ None (<class 'object'>,)


Exception: Out of range

In [5]:
import time
class timer():
    def __init__(self, message):
        self.message = message
        
    def __enter__(self):
        self.start = time.time()
        return None
    
    def __exit__(self, type, value, traceback):
        elapsed_time = (time.time() - self.start) * 1000
        print(self.message.format(elapsed_time))

In [6]:
class lazy_object:
    
    def __init__(self, callable, reset, *args, **kw):
        self.__dict__['callable'] = callable
        self.__dict__['args'] = args
        self.__dict__['kw'] = kw
        self.__dict__['obj'] = None
        self.reset = reset
        if self.reset == 1:
            del self.attr1
            
        
    def initObj(self):
        if self.obj is None:
            self.__dict__['obj'] = self.callable(*self.args, **self.kw)
            
    def __getattr__(self, name):
        self.initObj()
        return getattr(self.obj, name)
    
    def __setattr__(self, name, value):
        self.initObj()
        setattr(self.obj, name, value)
        
    def __len__(self):
        self.initObj()
        return len(self.obj)
    
    def __getitem__(self, idx):
        self.initObj()
        return self.obj[idx]


In [7]:
class A:
    def __init__(self, num_elem):
        self.attr1 = list(range(num_elem))

a = lazy_object(A, None, num_elem=10**5)

print(a.__dict__)

{'callable': <class '__main__.A'>, 'args': (), 'kw': {'num_elem': 100000}, 'obj': <__main__.A object at 0x7f3c4c15a2b0>}


In [10]:
print(a.reset)
with timer('Elapsed: {}ms'):
    print(1 in a.attr1)
    
with timer('Elapsed: {}ms'):
    print(42 in a.attr1)
    
a.reset = 1 
print(a.reset)
      
with timer('Elapsed: {}ms'):
    print(42 in a.attr1)

1
True
Elapsed: 0.3485679626464844ms
True
Elapsed: 0.23794174194335938ms
1
True
Elapsed: 0.031948089599609375ms


In [11]:
a = lazy_object(A, num_elem=10**5)
print(1 in a.attr1) # работает долго
print(42 in a.attr1) # работает быстро
a.reset = 1
print(42 in a.attr1) # работает долго


TypeError: __init__() missing 1 required positional argument: 'reset'